# Simple Flowsheet Example with Heat Exchangers

This notebook demonstrates how to assemble, initialize, and run a very simple flowsheet using steam and heat exchangers. This briefly demonstrates the basics of setting up a flowsheet.  See the documentation for a more complete description of the framework.

## Imports

Import all the Pyomo and IDAES modules, classes and functions needed for this example. In the imports below the ```delta_temperature_amtd_rule``` is a function that is used for a Pyomo rule to generate an expression for the averge temperature difference in a heat exchanger. This is used later to override the default LMTD calculation, making the example slightly more robust when experimenting with different inputs. 

In [1]:
import pyomo.environ as pyo    # Frequently used Pyomo classes and functions 
from pyomo.network import Arc # Class for objects used to connect ports, similar to a material stream
from idaes.core import FlowsheetBlock # IDAES flowsheet model class
from idaes.unit_models import Heater, HeatExchanger # Heater and HeatExchanger model classes
from idaes.unit_models.heat_exchanger import delta_temperature_amtd_rule # A delta T rule
from idaes.property_models import Iapws95ParameterBlock # Physical property parameter block class 
from IPython.display import display, Markdown

## Initialize DMF

Features of the DMF provide easy online access to documentation.  In the cell below initialize the DMF.

### Example Code

```python
from idaes.dmf import magics
%dmf init . create
```

In [2]:
from idaes.dmf import magics
%dmf init . create

Cannot create new configuration at ".": file "config.yaml" exists. Will try without "create" option

*Success!* Using workspace at "."

## Function to Display Material Contitions

This function will create a table of the material state in every state block.  This makes looking at results a little easier.

In [3]:
import pandas as pd
from idaes.core import StateBlock

def state_table(m):
    """
    Create a Pandas data frame that shows the material state in
    every state block.
    
    Args:
        m: a Pyomo model containing IDAES StateBlockBase objects
    
    Returns:
        A Pandas dataframe with material states
    """
    head = ["State Block", 
        "Flow (mol/s)",
        "Temperature (K)",
        "Pressure (Pa)",
        "Enthalpy (J/mol)",
        "Vapor Fraction"]
    st = pd.DataFrame(columns=head)
    j = 0
    for c in model.component_objects():
        if isinstance(c, StateBlock):
            for i in c:
                row = [c[i].name, 
                       pyo.value(c[i].flow_mol), 
                       pyo.value(c[i].temperature), 
                       pyo.value(c[i].pressure), 
                       pyo.value(c[i].enth_mol),
                       pyo.value(c[i].vapor_frac)]
                st.loc[j] = row
                j += 1
    return st

## Create a Flowsheet

The first step to creating a running IDAES model is to create a flowsheet object. In the code below the following steps should be performed.

* Create a Pyomo ConcreteModel (called "model")
* Add a steady-state flowsheet object to the model (attribute of "model" called "fs") 
* Add a property parameter block to the flowsheet (attribute of "model.fs" called "properties")

Parameter blocks contain parameters that are (usually) constant and common to all property methods of the same type. Sharing a common parameter block is more efficient and also easier to modify when doing things like parameter estimation.

When creating the Flowsheet model (or other idaes block models) The default argument takes a dictionary with configuration options for the model.  In this case the "dynamic" option should be set to False to create a steady-state flowsheet.


### Example Code

```python
model = pe.ConcreteModel()
model.fs = FlowsheetBlock(default={"dynamic": False})
model.fs.properties = Iapws95ParameterBlock()
```

In [4]:
model = pyo.ConcreteModel()
model.fs = FlowsheetBlock(default={"dynamic": False})
model.fs.properties = Iapws95ParameterBlock()

## Create a Heater and HeatExchanger Model

Next unit models can be added to the flowsheet. Here we need to create a Heater and HeatExchanger object called "model.fs.heater" and "model.fs.heat_exchanger".  For the heater, the only configuration option that needed is "property_package" which takes a property parameter block, in this case "model.fs.properties". The property method in this example uses molar enthalpy and pressure as the state variables.

The heat exchanger model is a little more complicated.  It has two sides and each side can use a different property package. The default LMTD rule is also going to be replaced with the AMTD rule which was imported in the first step.  The heat exchanger config dictionary is shown below. The "side_1" and "side_2" keys are each paired with a dictionary for configuring the two ControlVolume0Ds representing the two sides of the heat exchanger.  The "delta_temperature_rule" key is paired with a function that takes a HeatExchanger instance and a time index and returns a temperature difference expression.

```python
{"delta_temperature_rule":delta_temperature_amtd_rule,
 "side_1":{"property_package": model.fs.properties},
 "side_2":{"property_package": model.fs.properties}}
```

### Example Code

```python
model.fs.heater = Heater(default={"property_package": model.fs.properties})
model.fs.heat_exchanger = HeatExchanger(default={
        "delta_temperature_rule":delta_temperature_amtd_rule,
        "side_1":{"property_package": model.fs.properties},
        "side_2":{"property_package": model.fs.properties}})
```

In [5]:
model.fs.heater = Heater(default={"property_package": model.fs.properties})
model.fs.heat_exchanger = HeatExchanger(default={
        "delta_temperature_rule":delta_temperature_amtd_rule,
        "side_1":{"property_package": model.fs.properties},
        "side_2":{"property_package": model.fs.properties}})

## Getting Reference Documentation

The DMF provides easy access to IDAES framework reference documentation. For example:
```python
%dmf help HeatExchanger
```
will open a new browser tab showing the HeatExchanger class documentation. 

In [6]:
%dmf help HeatExchanger

2019-03-01 15:26:28,959 [INFO] idaes.dmf.help: find HTML docs for module=idaes.unit_models.heat_exchanger class=HeatExchanger on paths=['/home/jeslick/git/idaes-dev/docs/build/html']


No Sphinx docs found for "HeatExchanger"

## Fix Inlets and Initialize

Fix the inlets of the heater and heat exchanger and the heat duty for the heater. Initialize the units.

The heater model has the has one inlet called "inlet" and a heat duty called "heat_duty" that can be fixed resulting in 0 degrees of freedom for the model. The inlet and the heat duty are indexed by time, but since this is a steady-state model, only time zero exists.  The state variables for the inlet are "enth_mol", "flow_mol", and "pressure".  These state variables depend on the physical property package.  In this case a version of the IAPWS-95 properties for steam is being used, which uses enthalpy and pressure as state variables. 

Set the heater inlet to: molar enthalpy = 4000 J/mol, molar flow = 100 mol/s, and pressure = 101325 Pa.  Set the heater heat duty to 2000000 J/s.

The heat exchanger has two inlets "inlet_1" for "side_1" and "inlet_2" for "side_2".  Fix the inlets as shown in the table.

| variable    | inlet_1    | inlet_2    |
|-------------|------------|------------|
| enth_mole   | 4000 J/mol | 3000 J/mol |
| pressure    | 101325 Pa  | 101325 Pa  |
| flow_mol    | 100 mol/s  | 100 mol/s  |

Each unit model has an initialize method to get the model to a solvable state. Call the initialize methods for each unit.

### Example Heater Code

```python
# Fix both heat exchanger intlets and initialize
model.fs.heater.inlet[0].enth_mol.fix(4000)
model.fs.heater.inlet[0].flow_mol.fix(100)
model.fs.heater.inlet[0].pressure.fix(101325)
model.fs.heater.heat_duty[0].fix(2000000)

model.fs.heater.initialize()
```

### Example Heat Exchanger Code

```python
# Fix both heat exchanger intlets and initialize
model.fs.heat_exchanger.inlet_1[0].flow_mol.fix(100)
model.fs.heat_exchanger.inlet_1[0].pressure.fix(101325)
model.fs.heat_exchanger.inlet_1[0].enth_mol.fix(4000)
model.fs.heat_exchanger.inlet_2[0].flow_mol.fix(100)
model.fs.heat_exchanger.inlet_2[0].pressure.fix(101325)
model.fs.heat_exchanger.inlet_2[0].enth_mol.fix(3000)
model.fs.heat_exchanger.overall_heat_transfer_coefficent.fix(100)

model.fs.heat_exchanger.initialize()
```

In [7]:
# Fix heater inlets and heat duty then initialize unit
model.fs.heater.inlet.enth_mol.fix(4000)
model.fs.heater.inlet.flow_mol.fix(100)
model.fs.heater.inlet.pressure.fix(101325)
model.fs.heater.heat_duty[0].fix(2000000)

model.fs.heater.initialize()

In [8]:
# Fix both heat exchanger intlets and initialize
model.fs.heat_exchanger.inlet_1.flow_mol.fix(100)
model.fs.heat_exchanger.inlet_1.pressure.fix(101325)
model.fs.heat_exchanger.inlet_1.enth_mol.fix(4000)
model.fs.heat_exchanger.inlet_2.flow_mol.fix(100)
model.fs.heat_exchanger.inlet_2.pressure.fix(101325)
model.fs.heat_exchanger.inlet_2.enth_mol.fix(3000)
model.fs.heat_exchanger.overall_heat_transfer_coefficient.fix(100)

model.fs.heat_exchanger.initialize()

## Solve the Disconnected Flowsheet

The next step in this example is to make sure the disconnected flowsheet will solve.  To solve the model create a slover the solve like a normal Pyomo model.

### Example Code
```python
solver = pyo.SolverFactory('ipopt')
solver.solve(model, tee=True)
```

In [9]:
solver = pyo.SolverFactory('ipopt')

In [10]:
solver.solve(model, tee=True)

Ipopt 3.12.11: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.11, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:       21
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        9

Total number of variables............................:       11
                     variables with only lower bounds:        6
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       11
Total number of in

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 11, 'Number of variables': 11, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.12.11\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.06637787818908691}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

## Show States

Call the ```state_table(model)``` function written earlier to show a table of results. 

In [11]:
state_table(model)

,State Block,Flow (mol/s),Temperature (K),Pressure (Pa),Enthalpy (J/mol),Vapor Fraction
0,fs.heater.control_volume.properties_in[0.0],100,326.166708,101325,4000,0.000000
1,fs.heater.control_volume.properties_out[0.0],100,373.124296,101325,24000,0.404678
2,fs.heat_exchanger.side_1.properties_in[0.0],100,326.166708,101325,4000,0.000000
3,fs.heat_exchanger.side_1.properties_out[0.0],100,313.819218,101325,3070.04,0.000000
4,fs.heat_exchanger.side_2.properties_in[0.0],100,312.888963,101325,3000,0.000000
5,fs.heat_exchanger.side_2.properties_out[0.0],100,325.237048,101325,3929.96,0.000000


## Conncect the Flowsheet

The next step is to connect the outlet of the heater to the side_1 inlet of the heat exchanger. First unfix the varialbes in ```model.fs.heat_exchanger.inlet_1```. Also increase the flow rate of through side_2 of the heat exchanger to 600 mol/s.

Use a Pyomo Arc to connect the heater outlet to heat exchanger inlet_1 and apply the transforamtion to expand the arc to a set of constraints.

### Example Code

```python
model.fs.heat_exchanger.inlet_1.flow_mol.unfix()
model.fs.heat_exchanger.inlet_1.pressure.unfix()
model.fs.heat_exchanger.inlet_1.enth_mol.unfix()

model.fs.heat_exchanger.inlet_2.flow_mol.fix(600)

model.fs.stream_1 = Arc(source=model.fs.heater.outlet, 
                        destination=model.fs.heat_exchanger.inlet_1)

pyo.TransformationFactory("network.expand_arcs").apply_to(model)

```

In [12]:
model.fs.heat_exchanger.inlet_1.flow_mol.unfix()
model.fs.heat_exchanger.inlet_1.pressure.unfix()
model.fs.heat_exchanger.inlet_1.enth_mol.unfix()

model.fs.heat_exchanger.inlet_2.flow_mol.fix(600)

model.fs.stream_1 = Arc(source=model.fs.heater.outlet, 
                        destination=model.fs.heat_exchanger.inlet_1)

pyo.TransformationFactory("network.expand_arcs").apply_to(model)

## Solve the Connected Model

Solve the model again, and show results.

### Example Code

```python
solver.solve(model, tee=True)
state_table(model)
```

In [13]:
solver.solve(model, tee=True)
state_table(model)

Ipopt 3.12.11: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.11, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:       33
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       13

Total number of variables............................:       14
                     variables with only lower bounds:        8
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       14
Total number of in

,State Block,Flow (mol/s),Temperature (K),Pressure (Pa),Enthalpy (J/mol),Vapor Fraction
0,fs.heater.control_volume.properties_in[0.0],100,326.166708,101325,4000,0.000000
1,fs.heater.control_volume.properties_out[0.0],100,373.124296,101325,24000,0.404678
2,fs.heat_exchanger.side_1.properties_in[0.0],100,373.124296,101325,24000,0.404678
3,fs.heat_exchanger.side_1.properties_out[0.0],100,334.275568,101325,4611.18,0.000000
4,fs.heat_exchanger.side_2.properties_in[0.0],600,312.888963,101325,3000,0.000000
5,fs.heat_exchanger.side_2.properties_out[0.0],600,355.733262,101325,6231.47,0.000000


## Seeing Constraints

Sometimes you may want to see the exact equations that you are solving.  IDAES has some features to document model constraints.  First we will look at the constraints in the heat_excnger model. Between blocks duplicating symobols in the constraints is unavoidable, so each constraint is documented with its own symbol table and duplicate symbols within the constraints are indexed within angle brackets.  The fully qualified Pyomo component names are provided in the symbol tables.

In [14]:
from idaes.core.util.expr_doc import ipython_document_constraints

In [15]:
ipython_document_constraints(model.fs.heat_exchanger)

## fs.heat_exchanger.unit_heat_balance[0.0]
$$0.0 \le Q_{ 1 ,0.0 } + Q_{ 2 ,0.0 }\le 0.0$$

Symbol | Doc | Path
 ---: | --- | ---
**Variable** | **Doc** | **Path**
$Q_1$|Heat transfered in unit [J/s]|fs.heat_exchanger.side_1.heat
$Q_2$|Heat transfered in unit [J/s]|fs.heat_exchanger.side_2.heat

## fs.heat_exchanger.heat_transfer_equation[0.0]
$$0.0 \le - \Delta T_{ 0.0 } U_{ 0.0 } A + Q_{ 2 ,0.0 }\le 0.0$$

Symbol | Doc | Path
 ---: | --- | ---
**Variable** | **Doc** | **Path**
$Q_2$|Heat transfered in unit [J/s]|fs.heat_exchanger.side_2.heat
$U$|Overall heat transfer coefficient|fs.heat_exchanger.overall_heat_transfer_coefficient
$A$|Heat exchange area|fs.heat_exchanger.area
**Expression** | **Doc** | **Path**
$\Delta T$|Temperature difference driving force for heat transfer|fs.heat_exchanger.delta_temperature

## fs.heat_exchanger.side_1.material_balances[0.0,Mix,H2O]
$$0.0 \le F - F_{ D1 }\le 0.0$$

Symbol | Doc | Path
 ---: | --- | ---
**Variable** | **Doc** | **Path**
$F$|Total flow [mol/s]|fs.heat_exchanger.side_1.properties_in[0.0].flow_mol
$F_{ D1 }$|Total flow [mol/s]|fs.heat_exchanger.side_1.properties_out[0.0].flow_mol

## fs.heat_exchanger.side_1.enthalpy_balances[0.0]
$$0.0 \le 1.0 \cdot 10^{-6} h F - 1.0 \cdot 10^{-6} h_{ D1 } F_{ D1 } + 1.0 \cdot 10^{-6} Q_{ 1 ,0.0 }\le 0.0$$

Symbol | Doc | Path
 ---: | --- | ---
**Variable** | **Doc** | **Path**
$h$|Total molar enthalpy (J/mol)|fs.heat_exchanger.side_1.properties_in[0.0].enth_mol
$F$|Total flow [mol/s]|fs.heat_exchanger.side_1.properties_in[0.0].flow_mol
$h_{ D1 }$|Total molar enthalpy (J/mol)|fs.heat_exchanger.side_1.properties_out[0.0].enth_mol
$F_{ D1 }$|Total flow [mol/s]|fs.heat_exchanger.side_1.properties_out[0.0].flow_mol
$Q_1$|Heat transfered in unit [J/s]|fs.heat_exchanger.side_1.heat

## fs.heat_exchanger.side_1.pressure_balance[0.0]
$$0.0 \le 0.0001 P - 0.0001 P_{ D1 }\le 0.0$$

Symbol | Doc | Path
 ---: | --- | ---
**Variable** | **Doc** | **Path**
$P$|Pressure [Pa]|fs.heat_exchanger.side_1.properties_in[0.0].pressure
$P_{ D1 }$|Pressure [Pa]|fs.heat_exchanger.side_1.properties_out[0.0].pressure

## fs.heat_exchanger.side_2.material_balances[0.0,Mix,H2O]
$$0.0 \le F - F_{ D1 }\le 0.0$$

Symbol | Doc | Path
 ---: | --- | ---
**Variable** | **Doc** | **Path**
$F$|Total flow [mol/s]|fs.heat_exchanger.side_2.properties_in[0.0].flow_mol
$F_{ D1 }$|Total flow [mol/s]|fs.heat_exchanger.side_2.properties_out[0.0].flow_mol

## fs.heat_exchanger.side_2.enthalpy_balances[0.0]
$$0.0 \le 1.0 \cdot 10^{-6} h F - 1.0 \cdot 10^{-6} h_{ D1 } F_{ D1 } + 1.0 \cdot 10^{-6} Q_{ 2 ,0.0 }\le 0.0$$

Symbol | Doc | Path
 ---: | --- | ---
**Variable** | **Doc** | **Path**
$h$|Total molar enthalpy (J/mol)|fs.heat_exchanger.side_2.properties_in[0.0].enth_mol
$F$|Total flow [mol/s]|fs.heat_exchanger.side_2.properties_in[0.0].flow_mol
$h_{ D1 }$|Total molar enthalpy (J/mol)|fs.heat_exchanger.side_2.properties_out[0.0].enth_mol
$F_{ D1 }$|Total flow [mol/s]|fs.heat_exchanger.side_2.properties_out[0.0].flow_mol
$Q_2$|Heat transfered in unit [J/s]|fs.heat_exchanger.side_2.heat

## fs.heat_exchanger.side_2.pressure_balance[0.0]
$$0.0 \le 0.0001 P - 0.0001 P_{ D1 }\le 0.0$$

Symbol | Doc | Path
 ---: | --- | ---
**Variable** | **Doc** | **Path**
$P$|Pressure [Pa]|fs.heat_exchanger.side_2.properties_in[0.0].pressure
$P_{ D1 }$|Pressure [Pa]|fs.heat_exchanger.side_2.properties_out[0.0].pressure


## Named Expressions

Looking at the heat transfer equation above you see that $\Delta T$ is not a variable but a expression.  To find out how $\Delta T$ is calculated call ipython_document_constraints() again with the $\Delta T$ as the argument.  With a named expression at the top level, the constraint documenting function will descend into the expression.  Clos inspection will show that will show that this is the arethmetic average of the temperature differences at the two ends of the heat exchanger. 

In [16]:
ipython_document_constraints(model.fs.heat_exchanger.delta_temperature[0])

$$0.5 T - 0.5 T_{ D1 } + 0.5 T_{ D2 } - 0.5 T_{ D3 }$$

Symbol | Doc | Path
 ---: | --- | ---
**Expression** | **Doc** | **Path**
$\Delta T$|Temperature difference driving force for heat transfer|fs.heat_exchanger.delta_temperature
$T$|Temperature (K)|fs.heat_exchanger.side_1.properties_in[0.0].temperature
$T_{ D1 }$|Temperature (K)|fs.heat_exchanger.side_2.properties_out[0.0].temperature
$T_{ D2 }$|Temperature (K)|fs.heat_exchanger.side_1.properties_out[0.0].temperature
$T_{ D3 }$|Temperature (K)|fs.heat_exchanger.side_2.properties_in[0.0].temperature


## ExternalFunctions

The above results for $\Delta T$ show that even temperature itself is an expression. Temperature is an expression because the property method used in this example has molar enthalpy and pressure as state variables.  Temperature is actually computed by an external function. The external function returns $T_c/T$ as a function of enthalpy in kJ/kg and pressure in kPa, so there are some conversions built into the temperature expression. 

In [17]:
ipython_document_constraints(model.fs.heat_exchanger.side_1.properties_in[0].temperature)

$$\frac{647.096}{\operatorname{func_{0}}{\left (0.001 u_1,0.001 P \right )}}$$

Symbol | Doc | Path
 ---: | --- | ---
**Variable** | **Doc** | **Path**
$P$|Pressure [Pa]|fs.heat_exchanger.side_1.properties_in[0.0].pressure
**Expression** | **Doc** | **Path**
$T$|Temperature (K)|fs.heat_exchanger.side_1.properties_in[0.0].temperature
$u_1$|Mass enthalpy (J/kg)|fs.heat_exchanger.side_1.properties_in[0.0].enth_mass
**Function** | **Doc** | **Path**
$func_0$|None|fs.heat_exchanger.side_1.properties_in[0.0].func_tau
